# MXNet Model Server

This Notebook is borrowed from: https://github.com/TalkAI/facial-emotion-recognition-gluon/blob/master/notebooks/Gluon_FERPlus.ipynb

## Install the required Packages locally

We will need the PyPi packages listed below to test model server locally, and to perform image pre-processing prior to the model inference.

In [ ]:
!pip install -q mxnet-model-server==1.0
print('mxnet-model-server installed')
!pip install -q scikit-image==0.13.0
print('scikit-learn installed')
!pip install -q opencv-python
print('opencv-python installed')

In [ ]:
# We define the model's input and output type and shape via signature.json
%cat ./model_archive_resources/signature.json

In [ ]:
# We define the model's class label names via synset.txt
%cat ./model_archive_resources/synset.txt

In [ ]:
# And lastly, we define custom code for request handling via python code other auxiliary files
%cat ./model_archive_resources/fer_service.py

## Download Model

In [ ]:
!mkdir -p /home/ec2-user/SageMaker/mxnet-workshop-gaic/lab/ferplus
%pushd ~/SageMaker/mxnet-workshop-gaic/lab/ferplus
!curl -O https://s3.amazonaws.com/mxnet-demo-models/models/fer/gluon_ferplus-0000.params
!curl -O https://s3.amazonaws.com/mxnet-demo-models/models/fer/gluon_ferplus-symbol.json

!mv ~/SageMaker/mxnet-workshop-gaic/lab/ferplus/gluon_ferplus-0000.params ~/SageMaker/mxnet-workshop-gaic/lab/ferplus/ferplus-0000.params
!mv ~/SageMaker/mxnet-workshop-gaic/lab/ferplus/gluon_ferplus-symbol.json ~/SageMaker/mxnet-workshop-gaic/lab/ferplus/ferplus-symbol.json

## Export model

In [ ]:
%popd
%cp -r ./model_archive_resources/* ~/SageMaker/mxnet-workshop-gaic/lab/ferplus/

In [ ]:
%pushd ~/SageMaker/mxnet-workshop-gaic/lab/ferplus

In [ ]:
ls -al ./fer_service.py

In [ ]:
# Let's package everything up into a Model Archive bundle
!model-archiver -f --model-name ferplus --handler fer_service:handle --model-path /home/ec2-user/SageMaker/mxnet-workshop-gaic/lab/ferplus --export-path /home/ec2-user/SageMaker/mxnet-workshop-gaic/lab

In [ ]:
!ls -l ../ferplus.mar

In [ ]:
# Spawning a new process to run the server
import subprocess as sp
server = sp.Popen("mxnet-model-server --start --model-store /home/ec2-user/SageMaker/mxnet-workshop-gaic/lab --models ferplus=ferplus.mar", shell=True)

In [ ]:
server.pid

In [ ]:
# Check out the health endpoint
!curl http://localhost:8080/ping

In [ ]:
!ls

In [ ]:
%popd
test_image='happy.jpg'

In [ ]:
from matplotlib.pyplot import imshow
from PIL import Image
import numpy as np
img = Image.open(test_image)
imshow(np.asarray(img))

In [ ]:
!curl -X POST http://localhost:8080/predictions/ferplus -F "data=@$test_image"

In [ ]:
# Lastly, we'll terminate the server
server.terminate()